## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import json

# Import API key
from config import g_API_KEY
# import os
# from dotenv import load_dotenv
#Google_API_KEY = os.getenv("g_API_KEY")



In [2]:
# Configure gmaps API key
gmaps.configure(api_key= g_API_KEY)


In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
# print(city_data_df.columns)
# city_data_df = city_data_df.set_index('Unnamed: 0')
city_data_df.drop(columns="Unnamed: 0", inplace=True)
# city_data_df.drop(index = 0, inplace=True) # use onl.y to remove rows
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Puerto Ayora,EC,-0.7393,-90.3518,74.55,97,72,0.47,broken clouds
1,Constitucion,CL,-35.3333,-72.4167,58.64,92,66,1.77,broken clouds
2,Cidreira,BR,-30.1811,-50.2056,73.67,76,100,14.14,light rain
3,Carnarvon,AU,-24.8667,113.6333,86.07,54,40,24.16,scattered clouds
4,Punta Arenas,CL,-53.1500,-70.9167,46.51,71,20,19.57,few clouds


In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv", index_col = 0)
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Puerto Ayora,EC,-0.7393,-90.3518,74.55,97,72,0.47,broken clouds
1,Constitucion,CL,-35.3333,-72.4167,58.64,92,66,1.77,broken clouds
2,Cidreira,BR,-30.1811,-50.2056,73.67,76,100,14.14,light rain
3,Carnarvon,AU,-24.8667,113.6333,86.07,54,40,24.16,scattered clouds
4,Punta Arenas,CL,-53.1500,-70.9167,46.51,71,20,19.57,few clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
Min = float(input("What is the minimum temperature you would like for your trip? "))
Max = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 80


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filter_city_df = city_data_df.loc[(city_data_df['Max Temp']<=Max)
                                 & (city_data_df['Max Temp']>=Min)]
print(len(filter_city_df))
filter_city_df.head()


315


,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Puerto Ayora,EC,-0.7393,-90.3518,74.55,97,72,0.47,broken clouds
1,Constitucion,CL,-35.3333,-72.4167,58.64,92,66,1.77,broken clouds
2,Cidreira,BR,-30.1811,-50.2056,73.67,76,100,14.14,light rain
6,Busselton,AU,-33.6500,115.3333,75.60,60,90,14.65,overcast clouds
8,Necochea,AR,-38.5473,-58.7368,59.43,59,5,11.77,clear sky


In [7]:
# 4a. Determine if there are any empty rows.
filter_city_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 315 entries, 0 to 673
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 315 non-null    object 
 1   Country              309 non-null    object 
 2   Lat                  315 non-null    float64
 3   Lng                  315 non-null    float64
 4   Max Temp             315 non-null    float64
 5   Humidity             315 non-null    int64  
 6   Cloudiness           315 non-null    int64  
 7   Wind Speed           315 non-null    float64
 8   Current Description  315 non-null    object 
dtypes: float64(4), int64(2), object(3)
memory usage: 24.6+ KB


In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = filter_city_df.dropna()

clean_df.head()


,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Puerto Ayora,EC,-0.7393,-90.3518,74.55,97,72,0.47,broken clouds
1,Constitucion,CL,-35.3333,-72.4167,58.64,92,66,1.77,broken clouds
2,Cidreira,BR,-30.1811,-50.2056,73.67,76,100,14.14,light rain
6,Busselton,AU,-33.6500,115.3333,75.60,60,90,14.65,overcast clouds
8,Necochea,AR,-38.5473,-58.7368,59.43,59,5,11.77,clear sky


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,74.55,broken clouds,-0.7393,-90.3518,
1,Constitucion,CL,58.64,broken clouds,-35.3333,-72.4167,
2,Cidreira,BR,73.67,light rain,-30.1811,-50.2056,
6,Busselton,AU,75.60,overcast clouds,-33.6500,115.3333,
8,Necochea,AR,59.43,clear sky,-38.5473,-58.7368,
10,Rikitea,PF,79.97,broken clouds,-23.1203,-134.9692,
14,East London,ZA,65.23,moderate rain,-33.0153,27.9116,
20,Saint George,US,60.64,clear sky,37.1041,-113.5841,
21,Tocopilla,CL,69.31,scattered clouds,-22.0920,-70.1979,
22,Hermanus,ZA,65.48,scattered clouds,-34.4187,19.2345,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_API_KEY
}

# 6b. Iterate through the hotel DataFrame.
for i, city in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = city["Lat"]
    lng = city["Lng"]
    params["location"] = f'{lat}, {lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    print(f"Processing | {city['City']}")
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels= requests.get(base_url, params=params).json()
    #print(json.dumps(hotels, indent=4))
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[i, 'Hotel Name']= hotels['results'][0]['name']
    except:
        print(f"Hotel not found. Skipping {city['City']}...")

print("------Completed-------")

Processing | Puerto Ayora
Processing | Constitucion
Processing | Cidreira
Processing | Busselton
Processing | Necochea
Processing | Rikitea
Processing | East London
Processing | Saint George
Processing | Tocopilla
Processing | Hermanus
Processing | Victoria
Processing | Bryant
Processing | Kongolo
Processing | Hirara
Processing | Bluff
Processing | Avarua
Processing | Kapaa
Processing | Kaitangata
Processing | Katsuura
Processing | Aroeiras
Processing | Bredasdorp
Processing | Dong Hoi
Processing | Limon
Processing | Salinopolis
Processing | Kingston
Processing | Dunedin
Processing | Port Alfred
Processing | Cape Town
Processing | Ahipara
Processing | Chuy
Processing | Kruisfontein
Processing | Mataura
Processing | Tallahassee
Processing | Atuona
Processing | Hilo
Processing | Abha
Processing | Sungaipenuh
Processing | Thaba-Tseka
Processing | Mar Del Plata
Processing | Aripuana
Hotel not found. Skipping Aripuana...
Processing | Christchurch
Processing | Cabo San Lucas
Processing | Jut

In [11]:
print(len(hotel_df))
hotel_df

309


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,74.55,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
1,Constitucion,CL,58.64,broken clouds,-35.3333,-72.4167,Hotel Boutique Las Azucenas
2,Cidreira,BR,73.67,light rain,-30.1811,-50.2056,Hotel Castelo
6,Busselton,AU,75.60,overcast clouds,-33.6500,115.3333,Observatory Guest House
8,Necochea,AR,59.43,clear sky,-38.5473,-58.7368,dyd -mar Hotel
...,...,...,...,...,...,...,...
660,Niimi,JP,70.39,clear sky,34.9833,133.4667,備中路
662,Unai,BR,69.04,broken clouds,-16.3575,-46.9061,Hotel Regente
669,Mwene-Ditu,CD,68.56,overcast clouds,-7.0000,23.4500,Hôtel Kamas
670,Itarema,BR,74.70,overcast clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"


In [12]:
# 7. Drop the rows where there is no Hotel Name.
#clean_hotel_df = hotel_df.dropna()
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!=""]

print(len(clean_hotel_df))

clean_hotel_df

285


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,74.55,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
1,Constitucion,CL,58.64,broken clouds,-35.3333,-72.4167,Hotel Boutique Las Azucenas
2,Cidreira,BR,73.67,light rain,-30.1811,-50.2056,Hotel Castelo
6,Busselton,AU,75.60,overcast clouds,-33.6500,115.3333,Observatory Guest House
8,Necochea,AR,59.43,clear sky,-38.5473,-58.7368,dyd -mar Hotel
...,...,...,...,...,...,...,...
660,Niimi,JP,70.39,clear sky,34.9833,133.4667,備中路
662,Unai,BR,69.04,broken clouds,-16.3575,-46.9061,Hotel Regente
669,Mwene-Ditu,CD,68.56,overcast clouds,-7.0000,23.4500,Hôtel Kamas
670,Itarema,BR,74.70,overcast clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [23]:
# 11a. Add a marker layer for each city to the map. 
#max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30,31), zoom_level=1.5)
#heat_layer= gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             #max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))